In [6]:
import ase.db
db = ase.db.connect('/path/to/your/db')
rows = db.select()
all_atom = []
for row in rows:
    all_atom.append(row)

all_atom = all_atom[:100]

In [7]:
energies = [item.energy for item in all_atom]

In [8]:
# 高度分界线为11.5和13

def level_proportion(atoms):
    high_atoms = [atom for atom in atoms if atom.position[2] > 13.0]

    cr_count = sum(1 for atom in high_atoms if atom.symbol == 'Cr')

    proportion = cr_count / len(high_atoms)
    return proportion

def mid_proportion(atoms):
    high_atoms = [atom for atom in atoms if (atom.position[2] < 13.0) & (atom.position[2] > 11.5)]

    cr_count = sum(1 for atom in high_atoms if atom.symbol == 'Cr')

    proportion = cr_count / len(high_atoms)
    return proportion

def btm_proportion(atoms):
    high_atoms = [atom for atom in atoms if (atom.position[2] < 11.5)]

    cr_count = sum(1 for atom in high_atoms if atom.symbol == 'Cr')

    proportion = cr_count / len(high_atoms)
    return proportion

In [9]:
prop1 = [level_proportion(item.toatoms()) for item in all_atom]
prop2 = [mid_proportion(item.toatoms()) for item in all_atom]
prop3 = [btm_proportion(item.toatoms()) for item in all_atom]

In [10]:
from ase.neighborlist import neighbor_list
from collections import defaultdict
import numpy as np

def analyze_coordination(atoms):

    i, j, d = neighbor_list('ijd', atoms, cutoff=3.0) 

    element_types = sorted(set(atoms.get_chemical_symbols()))
    if len(element_types) != 3:
        raise ValueError("The structure must contain exactly three different elements.")


    coordination_counts = defaultdict(lambda: defaultdict(int))
    for index, neighbor_index in zip(i, j):
        element = atoms[index].symbol
        neighbor = atoms[neighbor_index].symbol
        coordination_counts[element][neighbor] += 1


    coordination_matrix = np.zeros((3, 3))


    for ix, element in enumerate(element_types):
        total_neighbors = sum(coordination_counts[element].values())
        for jx, neighbor in enumerate(element_types):
            if total_neighbors > 0:
                coordination_matrix[ix, jx] = coordination_counts[element][neighbor] / total_neighbors
            else:
                coordination_matrix[ix, jx] = 0

    return coordination_matrix, element_types


In [11]:
cn_matrix = [analyze_coordination(item.toatoms())[0] for item in all_atom]

In [ ]:
from ase.io import read
from ase.neighborlist import neighbor_list
import numpy as np

def calculate_cn_changes(atoms, a, b):
    """
    Calculate the ratio of coordination number (CN) changes within a given bond length range 
    for a specified atomic object.

    Parameters:
        atoms (ase.Atoms): An ASE Atoms object.
        a (float): Lower limit of the bond length.
        b (float): Upper limit of the bond length.
    
    Returns:
        cn_changes_ratio (float): The ratio of coordination number changes.
    """
    # Retrieve all neighbor pairs within the bond length range [a, b]
    i, j, d = neighbor_list('ijd', atoms, cutoff=b)
    
    # Filter distances within the specified range
    filtered_indices1 = (d < a)
    filtered_indices2 = (d > a) & (d <= b)
    i_filtered = i[filtered_indices1]
    j_filtered = j[filtered_indices2]
    
    # Calculate the coordination number for each atom within the range [a, b]
    cn_count = np.bincount(i_filtered, minlength=len(atoms))
    
    # Compute the ratio of coordination number changes
    initial_cn = np.mean(cn_count)
    final_cn = np.mean(np.bincount(j_filtered, minlength=len(atoms)))
    
    if initial_cn == 0:
        cn_changes_ratio = 0  # Avoid division by zero
    else:
        cn_changes_ratio = (final_cn - initial_cn) / initial_cn

    return cn_changes_ratio


In [13]:
from tqdm import tqdm

cell = all_atom[0].cell
b = np.linalg.norm(cell[0])
a = np.linalg.norm(cell[1])

pbar = tqdm(all_atom, total=len(all_atom))
cn_changes_ratios = [calculate_cn_changes(item.toatoms(), a, b) for item in pbar]

100%|██████████| 100/100 [01:20<00:00,  1.25it/s]
